In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() ## default dimension is 1
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        ## 예측과 손실 계산
        preds = model(X)
        loss = loss_fn(preds, y)

        ## 역전파 진행
        optimizer.zero_grad() ## 모든 매개변수들의 변화도를 0으로 저장하고 시작
        loss.backward() ## 예측 손실에 대한 역전파 진행. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
        optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수를 조정함

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct) : >0.1f}%, Avg loss : {test_loss:>8f} \n')

In [12]:
model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print('Done!')

Epoch 1
--------------------------
loss: 2.299848  [   64/60000]
loss: 2.289236  [ 6464/60000]
loss: 2.278111  [12864/60000]
loss: 2.272282  [19264/60000]
loss: 2.254996  [25664/60000]
loss: 2.224662  [32064/60000]
loss: 2.225282  [38464/60000]
loss: 2.190736  [44864/60000]
loss: 2.183239  [51264/60000]
loss: 2.158673  [57664/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss : 2.154599 

Epoch 2
--------------------------
loss: 2.157562  [   64/60000]
loss: 2.147158  [ 6464/60000]
loss: 2.095896  [12864/60000]
loss: 2.113971  [19264/60000]
loss: 2.064977  [25664/60000]
loss: 1.999688  [32064/60000]
loss: 2.018820  [38464/60000]
loss: 1.940015  [44864/60000]
loss: 1.944428  [51264/60000]
loss: 1.866601  [57664/60000]
Test Error: 
 Accuracy: 62.1%, Avg loss : 1.878058 

Epoch 3
--------------------------
loss: 1.907127  [   64/60000]
loss: 1.876403  [ 6464/60000]
loss: 1.768973  [12864/60000]
loss: 1.805123  [19264/60000]
loss: 1.701433  [25664/60000]
loss: 1.646374  [32064/60000]
loss: 1.6